In [ ]:
import torch
import einops
from einops import rearrange  # 修复拼写错误
from torch import Linear

class MHA(torch.nn.Module):
    def __init__(self,hs,heads):
        super().__init__()
        self.q_l=Linear(hs,hs)
        self.k_l=Linear(hs,hs)
        self.v_l=Linear(hs,hs)
        self.o_l=Linear(hs,hs)
        self.head_dim=hs//heads
        self.heads=heads
    
    def forward(self,x,mask=None):
        q=self.q_l(x)
        k=self.k_l(x)
        v=self.v_l(x)
        q=rearrange(q,"b seq_len (head h_dim) -> b head seq_len h_dim")  # 修复拼写错误
        k=rearrange(k,"b seq_len (head h_dim) -> b head seq_len h_dim")  # 修复拼写错误
        v=rearrange(v,"b seq_len (head h_dim) -> b head seq_len h_dim")  # 修复拼写错误
        attention_score=torch.matmul(q,k.transpose(-1,-2))/torch.sqrt(self.head_dim)  # 修复方法名
        # b head s s
        if mask is not None:  # 修复条件判断
            attention_score.masked_fill_(mask==0,float('-inf'))  # 修复方法名
        attention_prob=torch.softmax(attention_score, dim=-1)  # 建议添加维度参数
        # 对最后一个维度进行处理
        out=torch.matmul(attention_prob,v)  # 修复方法名
        # b head s h_dim
        out=rearrange(out,"b head seq_len head_dim -> b seq_len (head head_dim)")  # 修复拼写错误
        out=self.o_l(out)
        return out